In [1829]:
from sage.all import *
import os
from hashlib import shake_256
from random import random, randint

# **IMPLEMENTAÇÃO DO LPN**

In [1830]:
class LPN:
    def __init__(self, k, lambda_value, epsilon, seed=None):
        self.k = Integer(k)          
        self.lambda_value = Integer(lambda_value)    
        self.epsilon = Rational(epsilon)
        self.F2 = GF(2) 
        if seed is None:
            seed = os.urandom(16)
        self.master_seed = seed
        self.B = self.create_generator_B(self.master_seed)
        self.s = self.generate_B_lambda(self.k)

    def bernoulli_generator(self, epsilon, precision=64):
        w = vector(self.F2, [self.B() for _ in range(precision)])
        # Calcular o racional de Lebesgue w_chapeu
        w_chapeu = sum(Rational(w[i]) * Rational(2**(-(i+1))) for i in range(precision))
        # Retornar 1 se w_chapeu <= epsilon, 0 caso contrário
        return self.F2(1) if w_chapeu <= epsilon else self.F2(0)
            
    def create_generator_B(self, seed):
        xof_state = shake_256(seed)
        counter = [0]     
        def generator_B():
            bit_index = counter[0] % 8
            if bit_index == 0:
                new_bytes = xof_state.digest(1)
                generator_B.current_byte = new_bytes[0]
            bit = (generator_B.current_byte >> bit_index) & 1
            counter[0] += 1 
            return self.F2(bit)
        generator_B.current_byte = 0 
        return generator_B
        
    def generate_B_lambda(self, length):
        return vector(self.F2, [self.B() for _ in range(length)]) 
        
    def generate_sample(self):
        a = self.generate_B_lambda(self.k) 
        e = self.bernoulli_generator(self.epsilon)  
        t = self.F2(a.dot_product(self.s) + e) 
        return a, t
        
    def is_correct(self, a, t):
        expected = self.F2(a.dot_product(self.s))
        return t == expected
        
    def oracle(self):
        return self.generate_sample()

# **IMPLEMENTAÇÃO DO OT(N/N-1) SENDER PARA GRUPO FINITO P**

In [1831]:
class OTSenderFp:
    def __init__(self, N, k, lambda_value, epsilon, p):
        self.N = Integer(N) 
        self.k = Integer(k)
        self.lambda_value = Integer(lambda_value)
        self.epsilon = Rational(epsilon)
        self.F2 = GF(2)
        self.Fp = GF(p)
        self.seed = None
        self.ell = None
        self.a_u_pairs = None
        self.memory = {}
        
    def generate_xof_output(self, seed, ell):
        temp_lpn = LPN(self.k, self.lambda_value, self.epsilon, seed)
        a_u_pairs = []     
        for _ in range(ell):
            a = temp_lpn.generate_B_lambda(self.k)
            u = temp_lpn.B()
            a_u_pairs.append((a, u))   
        return a_u_pairs
    
    def setup(self, ell=128):
        """
            Inicializa o Sender e gera a informação inicial para o protocolo
        """
        self.seed = os.urandom(16)
        self.ell = Integer(ell)
        self.memory["seed"] = self.seed
        self.a_u_pairs = self.generate_xof_output(self.seed, ell)
        self.memory["a_u_pairs"] = self.a_u_pairs
        return self.seed, self.ell
    
    def verify_oblivious_criterion(self, t_vectors):
        """
         Verifica se o critério de oblivious é satisfeito pelos vetores t recebidos
        """
        if Integer(len(t_vectors)) != self.ell:
            return False        
            
        for i in range(self.ell):
            _, u_i = self.a_u_pairs[i]
            sum_t = self.F2(0)
            for k in range(self.N):
                sum_t += t_vectors[i][k]
                
            if sum_t != u_i:
                return False
                
        self.memory["t_vectors"] = t_vectors
        return True
    
    def fp_to_bits(self, fp_value):
        int_value = Integer(fp_value)
        bit_length = self.Fp.order().nbits()  # Tamanho correto
        return vector(self.F2, [self.F2((int_value >> i) & 1) for i in range(bit_length)])
    
    def bits_to_fp(self, bits):
        """
            Converte uma representação binária de volta para F_p
        """
        int_value = sum(Integer(bits[i]) << i for i in range(len(bits)))
        return self.Fp(int_value)
    
    
    
    def transfer(self, messages, num_iterations=11):
        """
            Gera os criptogramas para transferir mensagens em F_p^n
        """
        if Integer(len(messages)) != self.N:
            raise ValueError(f"Expected {self.N} messages, got {len(messages)}")
            
        if "t_vectors" not in self.memory:
            raise RuntimeError("Sender memory does not contain t_vectors")
            
        t_vectors = self.memory["t_vectors"]
        
        # Determinar o número de bits necessários para representar elementos em F_p
        bit_length = (self.Fp.order() - 1).nbits()
        
        all_cryptograms = []
        
        # Para cada mensagem em F_p, converter para bits e processar
        for message_idx in range(len(messages[0])):  # Assumindo que cada mensagem é um vetor em F_p^n
            # Extrair o elemento da posição message_idx de cada mensagem
            current_fp_elements = [msg[message_idx] for msg in messages]
            
            # Converter cada elemento de F_p para sua representação binária
            bit_representations = [self.fp_to_bits(element) for element in current_fp_elements]
            
            # Para cada posição de bit nas representações binárias
            for bit_pos in range(bit_length):
                # Extrair o bit na posição bit_pos de cada representação
                bit_messages = []
                for bits in bit_representations:
                    bit_value = bits[bit_pos] if bit_pos < len(bits) else self.F2(0)
                    bit_messages.append(bit_value)
                    
                # Gerar criptogramas para esta posição de bit
                bit_cryptograms = []
                max_hamming_weight = 32
                temp_lpn = LPN(self.k, self.lambda_value, self.epsilon, self.seed)
                
                for _ in range(num_iterations):
                    # Gerar vetor r com peso de Hamming limitado
                    r = None
                    while True:
                        r_candidate = vector(self.F2, [temp_lpn.B() for _ in range(self.ell)])
                        hw = sum(1 for bit in r_candidate if bit == self.F2(1))
                        if hw <= max_hamming_weight:
                            r = r_candidate
                            break
                            
                    # Calcular a'
                    a_prime = vector(self.F2, [self.F2(0) for _ in range(self.k)])
                    for i in range(self.ell):
                        if r[i] == self.F2(1):
                            a_prime += self.a_u_pairs[i][0]
                            
                    # Calcular c_k para cada mensagem (apenas o bit atual)
                    c = {}
                    for k in range(self.N):
                        c_k = bit_messages[k]  # Valor do bit atual
                        for i in range(self.ell):
                            if r[i] == self.F2(1):
                                c_k += t_vectors[i][k]
                        c[k] = c_k
                        
                    bit_cryptograms.append((a_prime, c))
                    
                # Adicionar os criptogramas desta posição de bit à lista completa
                all_cryptograms.append((message_idx, bit_pos, bit_cryptograms))
                
        return all_cryptograms

# **IMPLEMENTAÇÃO DO OT(N/N-1) RECEIVER PARA GRUPO FINITO P**

In [1832]:
class OTReceiverFp:
    def __init__(self, N, k, lambda_value, epsilon, p):
        self.N = Integer(N)
        self.k = Integer(k)
        self.lambda_value = Integer(lambda_value)
        self.epsilon = Rational(epsilon)
        self.F2 = GF(2)
        self.Fp = GF(p)  # Campo finito F_p
        self.seed = None
        self.ell = None
        self.a_u_pairs = None
        self.memory = {}
        
    def fp_to_bits(self, fp_value):
        int_value = Integer(fp_value)
        bit_length = self.Fp.order().nbits()  # Tamanho correto
        return vector(self.F2, [self.F2((int_value >> i) & 1) for i in range(bit_length)])
    
    def bits_to_fp(self, bits):
        """
            Converte uma representação binária de volta para F_p
        """
        int_value = sum(Integer(bits[i]) << i for i in range(len(bits)))
        return self.Fp(int_value)
    
    def generate_xof_output(self, seed, ell):
        temp_lpn = LPN(self.k, self.lambda_value, self.epsilon, seed)
        a_u_pairs = []     
        for _ in range(ell):
            a = temp_lpn.generate_B_lambda(self.k)
            u = temp_lpn.B()
            a_u_pairs.append((a, u))   
        return a_u_pairs
    
    def setup(self, seed, ell):
        """
            Configura o Receiver com o seed e ell recebidos do Sender
        """
        self.seed = seed
        self.ell = Integer(ell)
        self.a_u_pairs = self.generate_xof_output(seed, ell)
        self.memory["a_u_pairs"] = self.a_u_pairs   
    
    def choose(self, b):
        """
            Escolhe qual índice b (entre 0 e N-1) de mensagem excluir
        """
        if b < 0 or b >= self.N:
            raise ValueError(f"b must be in the range [0, {self.N-1}]")     
            
        self.memory["b"] = b
        
        secrets = {}
        temp_lpn = LPN(self.k, self.lambda_value, self.epsilon, self.seed)

        for k in range(self.N):
            if k != b:
                secrets[k] = temp_lpn.generate_B_lambda(self.k)
            else:
                secrets[k] = None
                
        self.memory["secrets"] = secrets
        
        t_vectors = []     
        for i in range(self.ell):
            a_i, u_i = self.a_u_pairs[i]
            t_i = {}
            sum_t = self.F2(0)

            for k in range(self.N):
                if k != b:
                    s_k = secrets[k]
                    # Gera o ruído usando Bernoulli
                    e_i_k = temp_lpn.bernoulli_generator(self.epsilon)
                    t_i_k = self.F2(a_i.dot_product(s_k) + e_i_k)
                    t_i[k] = t_i_k
                    sum_t += t_i_k
                    
            # Para k=b, garante que a soma de todos os t_i,k seja igual a u_i
            t_i[b] = self.F2(u_i - sum_t)     
            t_vectors.append(t_i)
            
        self.memory["t_vectors"] = t_vectors
        return t_vectors
        
    def decrypt(self, all_cryptograms, message_length):
        """
            Decifra os criptogramas multi-bit recebidos do Sender em F_p
        """
        if "b" not in self.memory or "secrets" not in self.memory:
            raise RuntimeError("Receiver memory does not contain choice b or secrets")
            
        b = self.memory["b"]
        secrets = self.memory["secrets"]
        
        
        bit_length = (self.Fp.order() - 1).nbits() # Determinar o número de bits necessários para representar elementos em F_p
        
        # Inicializar mensagens como vetores vazios
        final_messages = {k: [] for k in range(self.N) if k != b}
        
        # Organizar os criptogramas por elemento da mensagem
        cryptograms_by_element = {}
        for message_idx, bit_pos, bit_cryptograms in all_cryptograms:
            if message_idx not in cryptograms_by_element:
                cryptograms_by_element[message_idx] = {}
            cryptograms_by_element[message_idx][bit_pos] = bit_cryptograms
        
        # Para cada posição no vetor da mensagem
        for message_idx in range(message_length):
            # Inicializar bits para cada mensagem k
            bit_results = {k: {bit_pos: [] for bit_pos in range(bit_length)} for k in range(self.N) if k != b}
            
            if message_idx not in cryptograms_by_element:
                continue
                
            # Para cada posição de bit
            for bit_pos, bit_cryptograms in cryptograms_by_element[message_idx].items():
                # Decifrar cada criptograma para esta posição de bit
                for a_prime, c in bit_cryptograms:
                    for k in range(self.N):
                        if k != b:
                            m_k_bit = self.F2(c[k] + a_prime.dot_product(secrets[k]))
                            bit_results[k][bit_pos].append(m_k_bit)
            
            # Determinar o valor final deste elemento por votação majoritária bit a bit
            for k in range(self.N):
                if k != b:
                    # Determinar cada bit por votação majoritária
                    final_bits = []
                    for bit_pos in range(bit_length):
                        if bit_pos in bit_results[k] and bit_results[k][bit_pos]:
                            counts = {self.F2(0): 0, self.F2(1): 0}
                            for m in bit_results[k][bit_pos]:
                                counts[m] += 1
                            # O bit com mais votos é o correto
                            majority_bit = max(counts.items(), key=lambda x: x[1])[0]
                            final_bits.append(majority_bit)
                        else:
                            final_bits.append(self.F2(0))
                    
                    # Converter bits de volta para elemento de F_p
                    fp_element = self.bits_to_fp(final_bits)
                    final_messages[k].append(fp_element)
                        
        return final_messages

# **RUN_CODE PARA GRUPO FINITO P**

In [1833]:
class OTProtocolFp:
    def __init__(self, N, k, lambda_value, epsilon, p):
        self.N = Integer(N)
        self.k = Integer(k)
        self.lambda_value = Integer(lambda_value)
        self.epsilon = Rational(epsilon)
        self.p = Integer(p)
        self.sender = OTSenderFp(N, k, lambda_value, epsilon, p)
        self.receiver = OTReceiverFp(N, k, lambda_value, epsilon, p)
        self.Fp = GF(p)
        
    def run_protocol(self, b, messages, ell=128, num_iterations=11):
        """
            Executa o protocolo OT(N-1/N) para mensagens em F_p^n
        """

        print("🚩 Starting OT(N-1/N) protocol for messages in F_p^n...")
        
        # Verificar se as mensagens são vetores em F_p^n
        message_length = len(messages[0]) if messages else 0
        for msg in messages:
            if len(msg) != message_length:
                raise ValueError("All messages must have the same length")
            for element in msg:
                if element.parent() != self.Fp:
                    raise ValueError(f"Message elements must be in F_{self.p}")
        
        # Fase 1: Setup do Sender e escolha do Receiver
        print("➡️ Sender setup...")
        seed, ell = self.sender.setup(ell)
        
        print("⬅️ Passing parameters to Receiver...")
        self.receiver.setup(seed, ell)
        
        print(f"📌 Receiver chooses to exclude message index {b}...")
        t_vectors = self.receiver.choose(b)
        
        print("✅ Verifying oblivious criterion...")
        if not self.sender.verify_oblivious_criterion(t_vectors):
            print("❌ Error: Oblivious criterion failed")
            return None
            
        print(f"🔐 Sender transferring messages in F_{self.p}^{message_length}...")
        all_cryptograms = self.sender.transfer(messages, num_iterations)
        
        print("🔓 Receiver decrypting messages...")
        decrypted = self.receiver.decrypt(all_cryptograms, message_length)
        
        print("✅ Protocol completed successfully")
        return decrypted
    
    def run_protocol_without_prints(self, b, messages, ell=128, num_iterations=11):
        message_length = len(messages[0]) if messages else 0
        for msg in messages:
            if len(msg) != message_length:
                raise ValueError("All messages must have the same length")
            for element in msg:
                if element.parent() != self.Fp:
                    raise ValueError(f"Message elements must be in F_{self.p}")

        seed, ell = self.sender.setup(ell)
        self.receiver.setup(seed, ell)
        t_vectors = self.receiver.choose(b)
        if not self.sender.verify_oblivious_criterion(t_vectors):
            print("❌ Error: Oblivious criterion failed")
            return None
        all_cryptograms = self.sender.transfer(messages, num_iterations)
        decrypted = self.receiver.decrypt(all_cryptograms, message_length)
        return decrypted

# **Variáveis a ser usadas**

In [1834]:
p = 7  # primo para o campo finito F_p
N = 3  # número de mensagens
k = 4  # dimensão dos vetores secretos
lambda_value = 8  # parâmetro de segurança do LPN
epsilon = 0.1  # taxa de ruído
ell = 8  # número de amostras
num_iterations = 31  # número de iterações

# **"MAIN"**

In [1835]:
protocol = OTProtocolFp(N, k, lambda_value, epsilon, p)

b = randint(0, N - 1) # Escolha aleatória do índice a excluir

Fp = GF(p) # Criar mensagens como vetores em F_p^n

message_length = 3  # Vetores de comprimento 3
messages = [
    vector(Fp, [Fp(randint(0, p-1)) for _ in range(message_length)]) 
    for _ in range(N)
]

print(f"✉️ Original messages: {messages}")


decrypted = protocol.run_protocol(b, messages, ell, num_iterations)

if decrypted:
    print(f"📌 Receiver's choice to exclude: {b}")
    print(f"🔓 Decrypted messages: {decrypted}")

✉️ Original messages: [(1, 3, 0), (3, 4, 2), (6, 0, 5)]
🚩 Starting OT(N-1/N) protocol for messages in F_p^n...
➡️ Sender setup...
⬅️ Passing parameters to Receiver...
📌 Receiver chooses to exclude message index 1...
✅ Verifying oblivious criterion...
🔐 Sender transferring messages in F_7^3...
🔓 Receiver decrypting messages...
✅ Protocol completed successfully
📌 Receiver's choice to exclude: 1
🔓 Decrypted messages: {0: [1, 3, 0], 2: [6, 0, 5]}


--------------//----------------------------

# **COMEÇANDO O EXERCICIO 2**

--------------//----------------------------

## **2A**

In [1836]:
class SVOLE:
    def __init__(self, p, k, ell, lambda_value):
        """
        Inicializa os parâmetros do protocolo sVOLE com suporte para processamento elemento a elemento
        """
        self.p = Integer(p) 
        self.k = Integer(k) 
        self.ell = Integer(ell) 
        self.lambda_value = Integer(lambda_value) 
        self.N = self.lambda_value**2 + 2*self.lambda_value + 1
        
        # Definir os corpos finitos
        self.Fp = GF(p) # Corpo base F_p
        self.Fpk = self.Fp.extension(k) # Extensão F_p^k
        
        # Inicializar o protocolo OT
        self.epsilon = Rational(0.01) # Taxa de erro para LPN
        self.ot_protocol = OTProtocolFp(self.N, k, lambda_value, self.epsilon, p)
        
        self.Z = self._generate_Z_subset() # Gerar o subconjunto Z ⊂ F_p^k com N elementos

        self.Delta = None
        self.j = None
        
        self.current_index = 0  # Índice atual do elemento sendo processado
        self.x_elements = []  # Armazenar elementos à medida que são adicionados
        self.t_vectors = []  # Armazenar vetores t_i para cada elemento
        self.m_vector = []  # Armazenar valores m_i
        self.is_setup_done = False  # Flag para verificar se o setup foi concluído
        self.ot_messages = []  # Mensagens para o protocolo OT
        self.q_vector = []  # Armazenar valores q_i
    
    def _generate_Z_subset(self):
        """
        Gera o subconjunto Z ⊂ F_p^k com N elementos, garantindo todas as caracteristicas descritas
        """
        Fpk = self.Fpk
        Fp = self.Fp
        k = self.k
        while True:
            Z = []

            while len(Z) < self.N:
                elem = Fpk.random_element() # 𝑍≡{𝑧1,𝑧2,⋯,𝑧𝑁} ⊂ E
                if elem not in Z:
                    Z.append(elem)
            valid = True

            for i in range(self.N):
                basis_candidates = Z[:i] + Z[i+1:]
                vectors = []

                for elem in basis_candidates:
                    coeffs = elem.polynomial().list()
                    coeffs += [Fp(0)] * (k - len(coeffs))
                    vectors.append(vector(Fp, coeffs))

                mat = matrix(Fp, vectors).transpose()

                if mat.rank() != k: # Verificar se formam uma base
                    valid = False
                    break

            if valid:
                return Z
    
    def add_element(self, x_i):
        """
            Adiciona um elemento x_i ao protocolo sVOLE, gerando o vetor t_i e o valor m_i
        """
        if self.current_index >= self.ell:
            raise ValueError(f"Já foram adicionados {self.ell} elementos (máximo permitido)")
        
        x_i = self.Fp(x_i)
        self.x_elements.append(x_i)
        
        t_i = {}
        sum_t = self.Fp(0)
        for n in range(1, self.N):
            t_i[n] = self.Fp.random_element() 
            sum_t += t_i[n]
        t_i[0] = x_i - sum_t
        self.t_vectors.append(t_i)
        
        # Calcular m_i
        m_i = sum((-self.Z[n]) * t_i[n] for n in range(self.N))
        self.m_vector.append(self.Fpk(m_i))  # Converter para Fpk
        
        self.current_index += 1
        
        # Verificar se todos os elementos foram adicionados
        if self.current_index == self.ell:
            self._finalize_setup()
            return True
        else:
            return False
    
    def _finalize_setup(self):
        """
            Finaliza o setup após todos os elementos terem sido adicionados
        """
        # Preparar mensagens OT
        self.ot_messages = []
        for n in range(self.N):
            msg = [self.t_vectors[i][n] for i in range(self.ell)]
            self.ot_messages.append(msg)
        
        self.is_setup_done = True
    
    def verifier_setup(self):
        """
            Configura o Verifier, gerando a chave global (Δ)
        """
        if not self.is_setup_done:
            raise ValueError("É necessário adicionar todos os elementos antes de configurar o Verifier")
            
        self.j = randint(0, self.N - 1)  # Escolher aleatoriamente um índice j ∈ [N]
        return self.j
    
    def verifier_calculate_q(self, j, decrypted_messages):
        """
            Calcula o tag q para o Verifier
        """
        if not self.is_setup_done:
            raise ValueError("É necessário adicionar todos os elementos antes de calcular q")
        
        self.Delta = self.Z[j]  # Delta = z_j
         
        self.q_vector = []  # Calcular q_i para cada componente
        for i in range(self.ell):
            sum_part = sum((self.Delta - self.Z[n]) * decrypted_messages[n][i] for n in range(self.N) if n != j)
            q_i = sum_part
            self.q_vector.append(q_i)
        
        return self.q_vector, self.Delta
    
    def verify_relation(self):
        """
            Verifica a relação q = x * Delta + m
        """
        if not self.is_setup_done:
            raise ValueError("É necessário adicionar todos os elementos antes de verificar a relação")
            
        for i in range(self.ell):
            x_i = self.x_elements[i]
            m_i = self.m_vector[i]
            q_i = self.q_vector[i]
            
            if q_i != x_i * self.Delta + m_i:
                return False
        
        return True
    
    def run_incremental_protocol(self):
        """
            Executa o protocolo após todos os elementos terem sido adicionados
        """
        if not self.is_setup_done:
            raise ValueError("É necessário adicionar todos os elementos antes de executar o protocolo")

        print("🚩 Iniciando protocolo sVOLE incremental...")
        print("➡️ Configurando Verifier...")
        j = self.verifier_setup()

        print("🔄 Executando protocolo OT(N-1/N)...")
        decrypted = self.ot_protocol.run_protocol(j, self.ot_messages, self.ell)

        print("➡️ Verifier calculando tag q...")
        q_vector, Delta = self.verifier_calculate_q(j, decrypted)
        
        print("✅ Verificando a relação q = x * Delta + m...") 
        is_valid = self.verify_relation()
        

        print(f"🏁 Protocolo concluído: {'Válido' if is_valid else 'Inválido'}")
        
        return {
            "valid": is_valid,
            "Delta": Delta,
            "m_vector": self.m_vector,
            "q_vector": self.q_vector
        }
    
    def run_incremental_protocol_without_prints(self):
        if not self.is_setup_done:
            raise ValueError("É necessário adicionar todos os elementos antes de executar o protocolo")
        j = self.verifier_setup()
        decrypted = self.ot_protocol.run_protocol_without_prints(j, self.ot_messages, self.ell)
        q_vector, Delta = self.verifier_calculate_q(j, decrypted)
        is_valid = self.verify_relation()
        return {
            "valid": is_valid,
            "Delta": Delta,
            "m_vector": self.m_vector,
            "q_vector": self.q_vector
        }

In [1837]:
p = 7           # Primo pequeno para teste
k = 3           # Dimensão da extensão (k=2 para F_7^2)
ell = 3         # Dimensão do vetor x
lambda_value = 3 # Parâmetro de segurança do LPN
x_vector = [3, 1, 4] # Vetor de entrada para o Prover

In [1838]:
svole_incremental = SVOLE(p, k, ell, lambda_value)

print("\nAdicionando elementos um por um:")
for i, x_i in enumerate(x_vector):
    is_complete = svole_incremental.add_element(x_i)
    print(f"Adicionado elemento {i+1}: x_{i} = {x_i}, Completo: {is_complete}")

result_incremental = svole_incremental.run_incremental_protocol()

print("\nResultados (incremental):")
print(f"Delta = {result_incremental['Delta']}")
print(f"m_vector = {result_incremental['m_vector']}")
print(f"q_vector = {result_incremental['q_vector']}")
print(f"Autenticação: {'Sucesso' if result_incremental['valid'] else 'Falha'}")


Adicionando elementos um por um:
Adicionado elemento 1: x_0 = 3, Completo: False
Adicionado elemento 2: x_1 = 1, Completo: False
Adicionado elemento 3: x_2 = 4, Completo: True
🚩 Iniciando protocolo sVOLE incremental...
➡️ Configurando Verifier...
🔄 Executando protocolo OT(N-1/N)...
🚩 Starting OT(N-1/N) protocol for messages in F_p^n...
➡️ Sender setup...
⬅️ Passing parameters to Receiver...
📌 Receiver chooses to exclude message index 13...
✅ Verifying oblivious criterion...
🔐 Sender transferring messages in F_7^3...
🔓 Receiver decrypting messages...
✅ Protocol completed successfully
➡️ Verifier calculando tag q...
✅ Verificando a relação q = x * Delta + m...
🏁 Protocolo concluído: Válido

Resultados (incremental):
Delta = 4*z3
m_vector = [6*z3^2 + 3*z3 + 2, 6*z3^2 + z3 + 2, 6]
q_vector = [6*z3^2 + z3 + 2, 6*z3^2 + 5*z3 + 2, 2*z3 + 6]
Autenticação: Sucesso


In [1839]:
def run_k_times(times):
    """
    Executa o protocolo k vezes adicionando elementos incrementalmente e retorna o número de sucessos.
    """
    number_of_correct = 0
    for i in range(times):
        try:
            # Criar uma nova instância do protocolo
            svole = SVOLE(p, k, ell, lambda_value)
            # Gerar elementos aleatórios e adicioná-los um por um
            for _ in range(ell):
                x_i = svole.Fp.random_element()
                svole.add_element(x_i)
            
            # Executar o protocolo sem prints
            result = svole.run_incremental_protocol_without_prints()
            if result["valid"]:
                number_of_correct += 1
            else:
                print(f"❌ Falha na execução {i+1}")
        except Exception as e:
            print(f"❌ Erro na execução {i+1}: {e}")
    
    return number_of_correct


num_execucoes = 100
sucessos = run_k_times(num_execucoes)
print (f"✅ Número de execuções bem-sucedidas em {num_execucoes} tentativas: {sucessos}")

✅ Número de execuções bem-sucedidas em 100 tentativas: 100



| Uma forma de tornar isto quase sempre 100% é diminuir o lambda para 0.05 ou, até mesmo, 0.01 e aumentar o numero de iterações para 31 em vez de so 11. Contudo, para efeitos praticos, está bom o suficiente. 
----


# **2B**

In [1840]:
class ZK_SVOLE_PolySystem:
    def __init__(self, p, k, n, t, N, lambda_value, M=11):

        self.p = Integer(p)
        self.k = Integer(k)
        self.n = Integer(n)
        self.t = Integer(t)
        self.N = Integer(N)
        self.lambda_value = Integer(lambda_value)
        self.M = Integer(M)
        
        # Definir os corpos
        self.Fp = GF(p)
        self.Fpk = self.Fp.extension(k)  # Corpo estendido E = F_p^k
        
        # Armazenar as seeds e outros valores
        self.seeds = {}
        self.public_key = None
        self.private_key = None
        self.Z = None # Conjunto Z ⊂ E com |Z| = N
        
        # Parâmetro para o XOF
        self.epsilon = Rational(0.001)


        
    def _xof(self, seed, length):

        if isinstance(seed, list):
            seed_bytes  = bytes([int(''.join(map(str, seed[i:i+8])), 2) for i in range(0, len(seed), 8)])
        elif isinstance(seed, str):
            seed_bytes = seed.encode()
        elif isinstance(seed, bytes):
            seed_bytes = seed
        else:
            seed_bytes = str(seed).encode()
            
        xof = shake_256(seed_bytes)
        digest = xof.digest(length * 2)
        
        # Converte bytes para elementos em Fp
        elements = []
        for i in range(0, len(digest), 2):
            if i+1 < len(digest):
                value = (digest[i] << 8) | digest[i+1]
                elements.append(self.Fp(value % self.p))
            else:
                elements.append(self.Fp(digest[i] % self.p))
                
        return elements[:length] 
    

    
    def _generate_elements_from_seed(self, seed, count):

        elements = self._xof(seed, count)
        if len(elements) != count:
            raise ValueError(f"Gerados {len(elements)} elementos, mas {count} foram solicitados")
        
        return elements
    

    
    def _generate_vectors_from_seed(self, seed, num_vectors, vector_size):

        # Gerar todos os elementos de uma vez
        total_elements = num_vectors * vector_size
        all_elements = self._xof(seed, total_elements)
        
        # Dividir em vetores
        vectors = []
        for i in range(0, total_elements, vector_size):
            if i + vector_size <= total_elements:
                vectors.append(vector(self.Fp, all_elements[i:i+vector_size]))
        
        return vectors
    

    
    def _generate_Z_set(self, seed, N):
        Z = set()
        index = 0

        while len(Z) < N:
            coeffs = self._xof(str(seed) + str(index), self.k)
            element = self.Fpk(coeffs)

            Z.add(element)
            index += 1

        return list(Z)
    
        
    def KeyGen(self, lambda_value):

        rho = [randint(0, 1) for _ in range(lambda_value)]
        s = [randint(0, 1) for _ in range(lambda_value)]

        w_elements = self._generate_elements_from_seed(rho, self.n)
        w = vector(self.Fp, w_elements)

        if len(w) != self.n:
            raise ValueError(f"Vetor w tem dimensão {len(w)}, mas deveria ter {self.n}")

        # Gerar o conjunto Z ⊂ E com |Z| = N para o protocolo SVOLE
        self.Z = self._generate_Z_set(s, self.N)

        # Gerar todos os triplos usando {b_i, u_i, v_i}_{i∈[t]} ← XOF(s, 3 × n × t)
        total_elements = 3 * self.n * self.t
        all_vectors_elements = self._generate_elements_from_seed(s, total_elements)
        
        # Organizar os elementos em triplos de vetores (b_i, u_i, v_i)
        bs = []
        us = []
        vs = []
        c = []
        
        for i in range(self.t):
            # Extrair elementos para b_i
            start_b = i * 3 * self.n
            b_i = vector(self.Fp, all_vectors_elements[start_b:start_b + self.n])
            
            # Extrair elementos para u_i
            start_u = start_b + self.n
            u_i = vector(self.Fp, all_vectors_elements[start_u:start_u + self.n])
            
            # Extrair elementos para v_i
            start_v = start_u + self.n
            v_i = vector(self.Fp, all_vectors_elements[start_v:start_v + self.n])
            
            # v.b Calcular c_i = -(b_i * w) - (u_i * w) * (v_i * w)
            c_i = -(b_i.dot_product(w) + (u_i.dot_product(w) * v_i.dot_product(w)))
            
            # Armazenar os vetores e constantes
            bs.append(b_i)
            us.append(u_i)
            vs.append(v_i)
            c.append(c_i)
        
        # vi. Chave pública: (s, c) com c = {-c_i}_{i∈[t]}
        public_key = {'s': s, 'c': c}
        
        # Chave privada: w
        private_key = w
        
        # Armazenar as chaves e outros valores necessários
        self.public_key = public_key
        self.private_key = private_key
        self.seeds = {'rho': rho, 's': s}
        # deveria estar aqui? ou ser calculado quando preciso? Eu acho que nao, porque as contas que fazer para calcular o bs, us e vs sao tao triviais que seria so repetir o mesmo processo para obter o mesmo resultado
        self.bs = bs
        self.us = us
        self.vs = vs
        
        return public_key, private_key
    



    def _evaluate_polynomial(self, y, x, i):
        if not self.public_key:
            raise ValueError("As chaves não foram geradas ainda")
        
        c_i = self.public_key['c'][i]
        b_i = self.bs[i]
        u_i = self.us[i]
        v_i = self.vs[i]
        
        b_dot_x = sum(b_i[k] * x[k] for k in range(self.n))
        u_dot_x = sum(u_i[k] * x[k] for k in range(self.n))
        v_dot_x = sum(v_i[k] * x[k] for k in range(self.n))
    
        # Calcular f_i(y; x) = c_i + (b_i * x) * y + (u_i * x) * (v_i * x) --- (c_i já é negativo)
        result = c_i + (b_dot_x * y) + (u_dot_x * v_dot_x)
    
        return self.Fpk(result) 
    



    def Commit(self, w=None):

        if w is None:
            if self.private_key is None:
                raise ValueError("Chave privada não disponível. Execute KeyGen primeiro ou forneça w.")
            w = self.private_key
        else:
            w = vector(self.Fp, w)
        if len(w) != self.n:
            raise ValueError(f"w deve ter {self.n} elementos, mas tem {len(w)}")
        if not self.Z or len(self.Z) == 0:
            raise ValueError("O conjunto Z não foi gerado. Execute KeyGen primeiro.")
        

        # O Verifier gera aleatoriamente a chave global Δ ← Z
        j = randint(0, len(self.Z) - 1)
        Delta = self.Z[j]
        
        # Executar o protocolo sVOLE para a chave privada w
        svole_protocol = SVOLE(self.p, self.k, self.n + self.t, self.lambda_value)
        
        for k in range(self.n):
            svole_protocol.add_element(w[k])

        mu_values = [self.Fp.random_element() for _ in range(self.t)]
        mu = vector(self.Fp, mu_values)

        for i in range(self.t):
            svole_protocol.add_element(mu[i])

        auth_result = svole_protocol.run_incremental_protocol_without_prints()

        tau = vector(self.Fpk, auth_result["m_vector"][:self.n])
        omega = vector(self.Fpk, auth_result["q_vector"][:self.n])
        zeta = vector(self.Fpk, auth_result["m_vector"][self.n:])
        eta = vector(self.Fpk, auth_result["q_vector"][self.n:])
        Delta = auth_result["Delta"]

        for k in range(self.n):
            if omega[k] != w[k] * Delta + tau[k]:
                print(f"Verificação de ω falhou para k={k}")
        print(f"✅ passou ω")
    
        for i in range(self.t):
            if eta[i] != mu[i] * Delta + zeta[i]:
                print(f"Verificação de η falhou para i={i}")
        print(f"✅ passou η")
        
        # iv. Para cada polinômio {fᵢ}ᵢ∈[t], o Prover computa:
        # A₁,ᵢ ← bᵢ * τ + (uᵢ * w) * (vᵢ * τ) + (vᵢ * w) * (uᵢ * τ)
        # A₀,ᵢ ← (uᵢ * τ) * (vᵢ * τ)
        A1 = []
        A0 = []
        
        for i in range(self.t):
            # Obter os coeficientes do polinômio fᵢ
            b_i = self.bs[i] 
            u_i = self.us[i] 
            v_i = self.vs[i] 
            
            # Calcular os produtos internos necessários
            u_dot_tau = sum(u_i[k] * tau[k] for k in range(self.n))
            v_dot_tau = sum(v_i[k] * tau[k] for k in range(self.n))
            u_dot_w = sum(u_i[k] * w[k] for k in range(self.n))
            v_dot_w = sum(v_i[k] * w[k] for k in range(self.n))
            b_dot_tau = sum(b_i[k] * tau[k] for k in range(self.n))
            
            # A₁,ᵢ = bᵢ * τ + (uᵢ * w) * (vᵢ * τ) + (vᵢ * w) * (uᵢ * τ)
            A1_i = b_dot_tau + u_dot_w * v_dot_tau + v_dot_w * u_dot_tau
            
            # A₀,ᵢ = (uᵢ * τ) * (vᵢ * τ)
            A0_i = u_dot_tau * v_dot_tau
            
            A1.append(self.Fpk(A1_i))
            A0.append(self.Fpk(A0_i))
        
        # v. Para cada polinômio {fᵢ}ᵢ∈[t], o Verifier calcula Bᵢ ← fᵢ(Δ,ω)
        B = []
        for i in range(self.t):
            B_i = self._evaluate_polynomial(Delta, omega, i)
            B.append(B_i)
        
        # Montar o resultado do commit
        commit_results = {
            'Delta': Delta,
            'tau': tau,
            'omega': omega,
            'mu': mu,
            'zeta': zeta,
            'eta': eta,
            'A1': A1,
            'A0': A0,
            'B': B,
            'j': j  # Armazenar o índice j escolhido
        }
        
        print("✅ Commit concluído com sucesso!")
        
        return commit_results
    

    
    def Challenge(self, commit_results=None):
        # O Verifier gera uma seed aleatória
        if commit_results is None:
            if not hasattr(self, 'commit_results'):
                raise ValueError("Resultados do commit não disponíveis. Execute Commit primeiro ou forneça commit_results.")
            commit_results = self.commit_results
        else:
            # Armazenar para uso futuro
            self.commit_results = commit_results


        e = [randint(0, 1) for _ in range(self.lambda_value)]
        
        chi_elements = self._generate_elements_from_seed(e, self.t)
        chi = vector(self.Fp, chi_elements)
        
        B_default = sum(chi[i] * commit_results['B'][i] for i in range(self.t))
    
        # Calcular η* ← χ · η (η é o vetor eta calculado na Commit)
        eta_star = sum(chi[i] * commit_results['eta'][i] for i in range(self.t))

        # Calcular B_1 ← B* + η*
        B_star = B_default + eta_star

        # Montar o resultado do challenge
        challenge_results = {
            'e': e,
            'chi': chi,
            'B_default': B_default,
            'eta_star': eta_star,
            'B_star': B_star
        }
        
        return challenge_results
    

    
    def Prove(self, commit_results = None, challenge_results = None):

        if commit_results is None:
            if not hasattr(self, 'commit_results'):
                raise ValueError("Resultados do commit não disponíveis. Execute Commit primeiro ou forneça commit_results.")
            commit_results = self.commit_results
    
        if challenge_results is None:
            if not hasattr(self, 'challenge_results'):
                raise ValueError("Resultados do challenge não disponíveis. Execute Challenge primeiro ou forneça challenge_results.")
            challenge_results = self.challenge_results
        else:
            # Armazenar para uso futuro
            self.challenge_results = challenge_results
        
        e = challenge_results['e']
        chi = challenge_results['chi']

        A1_star = sum(chi[i] * commit_results['A1'][i] for i in range(self.t))
        A0_star = sum(chi[i] * commit_results['A0'][i] for i in range(self.t))

        # μ* ← χ · μ e ζ* ← χ · ζ
        mu_star = sum(chi[i] * commit_results['mu'][i] for i in range(self.t))
        zeta_star = sum(chi[i] * commit_results['zeta'][i] for i in range(self.t))

        # A₁* ← A* + μ* e A₀* ← A₀* + ζ*
        A1_final = A1_star + mu_star
        A0_final = A0_star + zeta_star

        # Montar a prova
        proof = {
            'A1_star': A1_final,
            'A0_star': A0_final
        }
    
        print("✅ Prova calculada com sucesso!")
    
        return proof
    
        
    
    def Verify(self, proof, commit_results=None, challenge_results=None):

        if commit_results is None:
            if not hasattr(self, 'commit_results'):
                raise ValueError("Resultados do commit não disponíveis.")
            commit_results = self.commit_results
    
        if challenge_results is None:
            if not hasattr(self, 'challenge_results'):
                raise ValueError("Resultados do challenge não disponíveis.")
            challenge_results = self.challenge_results

    
        # Extrair valores necessários
        A1_star = proof['A1_star']
        A0_star = proof['A0_star']
        B_star = challenge_results['B_star']
        Delta = commit_results['Delta']

        # Verificar B_1 =? A1_star · Δ + A0_star
        verification_result = (B_star == A1_star * Delta + A0_star)
    
        if verification_result:
            print("✅ Verificação bem-sucedida! A prova é válida.")
        else:
            print("❌ Verificação falhou! A prova não é válida.")
    
        return verification_result
    

    
    def run_protocol(self, w=None):
        if not self.public_key:
            raise ValueError("As chaves não foram geradas. Execute KeyGen primeiro.")

        print("🚩 Iniciando protocolo ZK-sVOLE completo...")

        # Fase de Commit
        commit_results = self.Commit(w)
        self.commit_results = commit_results

        # Fase de Challenge
        challenge_results = self.Challenge(commit_results)
        self.challenge_results = challenge_results

        # Fase de Prove
        proof = self.Prove(commit_results, challenge_results)

        # Fase de Verify
        is_valid = self.Verify(proof, commit_results, challenge_results)

        protocol_results = {
            'commit': commit_results,
            'challenge': challenge_results,
            'proof': proof,
            'is_valid': is_valid
        }

        print(f"🏁 Protocolo ZK-sVOLE concluído: {'Válido' if is_valid else 'Inválido'}")

        return protocol_results

In [1841]:
p = 7        # Característica do corpo primo
k = 3        # Dimensão da extensão do corpo
n = 3        # Número de variáveis
t = 5        # Número de polinômios
N = 11       # Número de elementos na base Z
lambda_value = 3  # Parâmetro de segurança

# Inicializar o protocolo
zk_svole = ZK_SVOLE_PolySystem(p, k, n, t, N, lambda_value)

# Gerar as chaves
public_key, private_key = zk_svole.KeyGen(lambda_value)
print(f"Chave privada: {private_key}")
print(f"Chave pública c: {public_key['c']}")

# Executar o protocolo usando a chave privada correta
result = zk_svole.run_protocol()
print(f"\nResultado do protocolo: {'Sucesso' if result['is_valid'] else 'Falha'}")


Chave privada: (5, 2, 6)
Chave pública c: [2, 0, 6, 4, 2]
🚩 Iniciando protocolo ZK-sVOLE completo...
✅ passou ω
✅ passou η
✅ Commit concluído com sucesso!
✅ Prova calculada com sucesso!
✅ Verificação bem-sucedida! A prova é válida.
🏁 Protocolo ZK-sVOLE concluído: Válido

Resultado do protocolo: Sucesso


In [1842]:
# Verificar com uma chave inválida
wrong_key = vector(GF(p), [1, 2, 3])  # Chave diferente
try:
    wrong_result = zk_svole.run_protocol(wrong_key)
    print(f"Resultado com chave inválida: {'Sucesso' if wrong_result['is_valid'] else 'Falha (esperado)'}")
except Exception as e:
    print(f"Erro durante a execução com chave inválida: {str(e)}")

🚩 Iniciando protocolo ZK-sVOLE completo...
✅ passou ω
✅ passou η
✅ Commit concluído com sucesso!
✅ Prova calculada com sucesso!
❌ Verificação falhou! A prova não é válida.
🏁 Protocolo ZK-sVOLE concluído: Inválido
Resultado com chave inválida: Falha (esperado)
